In [1]:
import datetime as dt
import random
import tensorflow as tf

# Get timestamp
timestamp = dt.datetime.now().strftime('%Y%m%d%H%M%S')

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.001
training_epochs = 8000
batch_size = 100
display_step = 10

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# tf graph input
X = tf.placeholder(tf.float32, [None, 784], name = 'X-input')
Y = tf.placeholder(tf.float32, [None, 10], name = 'Y-input')

# Set model weights
W1 = tf.get_variable("Weight1", shape = [784, 256], initializer = tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable("Weight2", shape = [256, 256], initializer = tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable("Weight3", shape = [256, 10], initializer = tf.contrib.layers.xavier_initializer())

B1 = tf.Variable(tf.random_normal([256]), name = 'Bias1')
B2 = tf.Variable(tf.random_normal([256]), name = 'Bias2')
B3 = tf.Variable(tf.random_normal([10]), name = 'Bias3')

tf.histogram_summary('Weight1', W1)
tf.histogram_summary('Weight2', W2)
tf.histogram_summary('Weight3', W3)

tf.histogram_summary('Bias1', B1)
tf.histogram_summary('Bias2', B2)
tf.histogram_summary('Bias3', B3)

<tf.Tensor 'HistogramSummary_5:0' shape=() dtype=string>

In [3]:
# Construct model
with tf.name_scope('Layer1'):
    L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), B1))
with tf.name_scope('Layer2'):
    L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), B2))
with tf.name_scope('Layer3'):
    hypothesis = tf.add(tf.matmul(L2, W3), B3)

# Minimize error using cross entropy
with tf.name_scope('Cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, Y))
    tf.scalar_summary('Cost', cost)

# Gradient Descent
with tf.name_scope('Train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1)), tf.float32))
    tf.scalar_summary('Accuracy', accuracy)

In [4]:
# Launch the graph
with tf.Session() as sess:
    writer = tf.train.SummaryWriter('/logs/mnist/nn_xavier_' + timestamp, sess.graph)
    merged = tf.merge_all_summaries()
    
    tf.initialize_all_variables().run()

    # Training cycle
    for epoch in range(training_epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # Fit training using batch data
        _, summary = sess.run([optimizer, merged], feed_dict = {X: batch_xs, Y: batch_ys})

        # Summarize
        writer.add_summary(summary, epoch)

    writer.close()
    print('Accuracy', accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))

Accuracy 0.9793
